In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles


In [ ]:
!pip install -U scikit-learn
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.9 MB/s eta 0:00:00


In [ ]:
# import all of the packages that will be used

# basic packages for data processing
import pandas as pd
import numpy as np
import gc

# packege for calculation
import math

# packages for machine learning
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

import xgboost

# packages to interpret the training result
import torch
from torchmetrics.classification import Accuracy, ConfusionMatrix, AUROC, Precision, Recall, F1Score

# package to benchmark the CPU running time
from time import process_time

# silent the warning from sklearn package
# import warnings
# warnings.filterwarnings('ignore')

# set the random seed to ensure the result is reproducible
import random
random.seed(42)
np.random.seed(42)

In [ ]:
fdt_hb_BABESG2_train = pd.read_csv('Ground_truth_datasets_characterised/Splits/Train/BABESG2_hybrid_characterised_index_output_train.csv')
fdt_hb_BABESG2_test = pd.read_csv('Ground_truth_datasets_characterised/Splits/Test/BABESG2_hybrid_characterised_index_output_test.csv')

In [ ]:
fdt_hb_BABESG2_train.columns

Index(['text', 'bias', 'bias_BABESG2', 'objective_BABESG2',
       'partly-subjective_BABESG2', 'subjective_BABESG2',
       'nonpartisan_headline', 'slight-partisan_headline',
       'hyperpartisan_headline', 'nonpartisan_article',
       'slight-partisan_article', 'hyperpartisan_article', 'true_LIAR',
       'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR',
       'pants-fire_LIAR', 'bluff_FakeNewsTFG', 'negative_NewsMTSC',
       'neutral_NewsMTSC', 'positive_NewsMTSC'],
      dtype='object')

In [ ]:
FEATURES = ['bias_BABESG2', 'objective_BABESG2',
       'partly-subjective_BABESG2', 'subjective_BABESG2',
       'nonpartisan_headline', 'slight-partisan_headline',
       'hyperpartisan_headline', 'nonpartisan_article',
       'slight-partisan_article', 'hyperpartisan_article', 'true_LIAR',
       'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR',
       'pants-fire_LIAR', 'bluff_FakeNewsTFG', 'negative_NewsMTSC',
       'neutral_NewsMTSC', 'positive_NewsMTSC']
TARGET = ['bias']

In [ ]:
def runKFold(clf, output_type, f = 5, scaler = False):

  scaler = StandardScaler()

  val_acc = []
  val_prec = []
  val_rec = []
  val_f1 = []
  val_auroc = []

  for k in range(1, 6):
    fdt_hb_BABESG2_train_K = pd.read_csv(f'Ground_truth_datasets_characterised/Splits/Train/BABESG2_hybrid_characterised_{output_type}_output_train_KFold{k}.csv')
    fdt_hb_BABESG2_val_K = pd.read_csv(f'Ground_truth_datasets_characterised/Splits/Validation/BABESG2_hybrid_characterised_{output_type}_output_val_KFold{k}.csv')

    train_X = fdt_hb_BABESG2_train_K[FEATURES]
    val_X = fdt_hb_BABESG2_val_K[FEATURES]
    train_y = fdt_hb_BABESG2_train_K['bias']
    val_y = fdt_hb_BABESG2_val_K['bias']

    if(scaler == True):
      scaler.fit(train_X)
      train_X = scaler.transform(train_X)
      train_X = scaler.transform(val_X)

    trained_clf = clf.fit(train_X, train_y)

    preds = trained_clf.predict(val_X)
    preds = torch.from_numpy(np.array(preds))

    val_y = torch.from_numpy(val_y.to_numpy())

    accuracy = Accuracy(task = 'binary', threshold=0.5)
    precision = Precision(task = 'binary', threshold=0.5)
    recall = Recall(task = 'binary', threshold=0.5)
    f1 = F1Score(task = 'binary', threshold=0.5)
    auroc = AUROC(task = 'binary')

    val_acc.append(accuracy(preds, val_y).item())
    val_prec.append(precision(preds, val_y).item())
    val_rec.append(recall(preds, val_y).item())
    val_f1.append(f1(preds, val_y).item())
    val_auroc.append(auroc(preds, val_y).item())

    k += 1

    gc.collect()

  return val_acc, val_prec, val_rec, val_f1, val_auroc


# Logistic Regression

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_LR = LogisticRegression(random_state = 42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_LR, 'index', 5, True)

In [ ]:
val_acc

[0.7941747307777405,
 0.7723735570907593,
 0.768482506275177,
 0.7801556587219238,
 0.7470816969871521]

In [ ]:
val_prec

[0.8181818127632141,
 0.753731369972229,
 0.7670682668685913,
 0.7957446575164795,
 0.7401574850082397]

In [ ]:
val_rec

[0.747035562992096,
 0.7984189987182617,
 0.7579365372657776,
 0.7420634627342224,
 0.7460317611694336]

In [ ]:
val_f1

[0.7809917330741882,
 0.7754318714141846,
 0.7624750733375549,
 0.767967164516449,
 0.7430830001831055]

In [ ]:
val_auroc

[0.7933651208877563,
 0.7727727293968201,
 0.7682812213897705,
 0.7794287204742432,
 0.7470616698265076]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7724536299705506
5-Fold CV Precision: 0.7749767184257508
5-Fold CV Recall: 0.7582972645759583
5-Fold CV F1 Score: 0.7659897685050965
5-Fold CV AUROC: 0.7721818923950196


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.015431388048695975
5-Fold CV Precision: 0.02836996624429412
5-Fold CV Recall: 0.020739761297537494
5-Fold CV F1 Score: 0.013078191467797966
5-Fold CV AUROC: 0.015151472477160666


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-22-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-22-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.794175,0.818182,0.747036,0.780992,0.793365
1,2,0.772374,0.753731,0.798419,0.775432,0.772773
2,3,0.768483,0.767068,0.757937,0.762475,0.768281
3,4,0.780156,0.795745,0.742063,0.767967,0.779429
4,5,0.747082,0.740157,0.746032,0.743083,0.747062
5,cv mean,0.772454,0.774977,0.758297,0.765990,0.772182
6,cv std,0.015431,0.028370,0.020740,0.013078,0.015151


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_LR = LogisticRegression(random_state = 42)

In [ ]:
trained_clf = clf_LR.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[440, 114],
        [109, 439]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7976406812667847


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7938517332077026 
Recall: 0.8010948896408081 
F1 Score: 0.7974568605422974


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7976593971252441


# Linear Discriminant Analysis (LDA)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_LDA = LinearDiscriminantAnalysis()

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_LDA, 'index', 5, scaler = True)

In [ ]:
val_acc

[0.7864077687263489,
 0.7607004046440125,
 0.768482506275177,
 0.774319052696228,
 0.7490272521972656]

In [ ]:
val_prec

[0.8206278085708618,
 0.7425373196601868,
 0.7692307829856873,
 0.8035714030265808,
 0.7430830001831055]

In [ ]:
val_rec

[0.7233201861381531,
 0.7865612506866455,
 0.7539682388305664,
 0.7142857313156128,
 0.7460317611694336]

In [ ]:
val_f1

[0.7689075469970703,
 0.7639155387878418,
 0.7615230679512024,
 0.756302535533905,
 0.7445544600486755]

In [ ]:
val_auroc

[0.7853242754936218,
 0.761096715927124,
 0.7682055234909058,
 0.7731733918190002,
 0.748970091342926]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7677873969078064
5-Fold CV Precision: 0.7758100628852844
5-Fold CV Recall: 0.7448334336280823
5-Fold CV F1 Score: 0.759040629863739
5-Fold CV AUROC: 0.7673539996147156


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.012585963927640229
5-Fold CV Precision: 0.03162513346568113
5-Fold CV Recall: 0.025385323734751983
5-Fold CV F1 Score: 0.008302163164649774
5-Fold CV AUROC: 0.01212141011760251


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-46-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-46-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.786408,0.820628,0.723320,0.768908,0.785324
1,2,0.760700,0.742537,0.786561,0.763916,0.761097
2,3,0.768483,0.769231,0.753968,0.761523,0.768206
3,4,0.774319,0.803571,0.714286,0.756303,0.773173
4,5,0.749027,0.743083,0.746032,0.744554,0.748970
5,cv mean,0.767787,0.775810,0.744833,0.759041,0.767354
6,cv std,0.012586,0.031625,0.025385,0.008302,0.012121


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train.drop(['text', 'bias'], axis=1)
test_X = fdt_hb_BABESG2_test.drop(['text', 'bias'], axis=1)

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_LDA = LinearDiscriminantAnalysis()

In [ ]:
trained_clf = clf_LDA.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[444, 110],
        [108, 440]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8021778464317322


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.800000011920929 
Recall: 0.8029196858406067 
F1 Score: 0.8014571666717529


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8021818399429321


# Quadratic Discriminant Analysis (QDA)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_QDA = QuadraticDiscriminantAnalysis()

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_QDA, 'index', 5, True)

/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [ ]:
val_acc

[0.7922329902648926,
 0.7782101035118103,
 0.7782101035118103,
 0.7665369510650635,
 0.7568093538284302]

In [ ]:
val_prec

[0.811965823173523,
 0.7642585635185242,
 0.7804877758026123,
 0.7727272510528564,
 0.7490196228027344]

In [ ]:
val_rec

[0.7509881258010864,
 0.7944663763046265,
 0.761904776096344,
 0.7420634627342224,
 0.7579365372657776]

In [ ]:
val_f1

[0.7802874445915222,
 0.7790697813034058,
 0.7710843086242676,
 0.7570850253105164,
 0.7534517049789429]

In [ ]:
val_auroc

[0.7915245294570923,
 0.7784592509269714,
 0.777898907661438,
 0.7660698890686035,
 0.7568308711051941]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7743999004364014
5-Fold CV Precision: 0.77569180727005
5-Fold CV Recall: 0.7614718556404114
5-Fold CV F1 Score: 0.768195652961731
5-Fold CV AUROC: 0.7741566896438599


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.011985889124851589
5-Fold CV Precision: 0.02092710502881827
5-Fold CV Recall: 0.017820878746781614
5-Fold CV F1 Score: 0.011078199063650242
5-Fold CV AUROC: 0.01182994861056344


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-70-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-70-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.792233,0.811966,0.750988,0.780287,0.791525
1,2,0.778210,0.764259,0.794466,0.779070,0.778459
2,3,0.778210,0.780488,0.761905,0.771084,0.777899
3,4,0.766537,0.772727,0.742063,0.757085,0.766070
4,5,0.756809,0.749020,0.757937,0.753452,0.756831
5,cv mean,0.774400,0.775692,0.761472,0.768196,0.774157
6,cv std,0.011986,0.020927,0.017821,0.011078,0.011830


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_QDA = QuadraticDiscriminantAnalysis()

In [ ]:
trained_clf = clf_QDA.fit(train_X_scaled, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[445, 109],
        [ 96, 452]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8139746189117432


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8057041168212891 
Recall: 0.8248175382614136 
F1 Score: 0.8151487708091736


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8140333890914917


# Gaussian Naive Bayes (GaussianNB)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8194174766540527,
 0.801556408405304,
 0.7801556587219238,
 0.7918288111686707,
 0.7645914554595947]

In [ ]:
val_prec

[0.8030303120613098,
 0.7937743067741394,
 0.7642585635185242,
 0.7799227833747864,
 0.7471697926521301]

In [ ]:
val_rec

[0.8379446864128113,
 0.8063241243362427,
 0.7976190447807312,
 0.8015872836112976,
 0.7857142686843872]

In [ ]:
val_f1

[0.8201160430908203,
 0.800000011920929,
 0.7805825471878052,
 0.790606677532196,
 0.7659574747085571]

In [ ]:
val_auroc

[0.8197357058525085,
 0.8016294836997986,
 0.780488908290863,
 0.7920150756835938,
 0.7649945020675659]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7915099620819092
5-Fold CV Precision: 0.777631151676178
5-Fold CV Recall: 0.805837881565094
5-Fold CV F1 Score: 0.7914525508880615
5-Fold CV AUROC: 0.7917727351188659


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.01861928205720912
5-Fold CV Precision: 0.020088259696921084
5-Fold CV Recall: 0.01744511396344121
5-Fold CV F1 Score: 0.018240209795627262
5-Fold CV AUROC: 0.018566316096747238


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-94-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-94-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.819417,0.803030,0.837945,0.820116,0.819736
1,2,0.801556,0.793774,0.806324,0.800000,0.801629
2,3,0.780156,0.764259,0.797619,0.780583,0.780489
3,4,0.791829,0.779923,0.801587,0.790607,0.792015
4,5,0.764591,0.747170,0.785714,0.765957,0.764995
5,cv mean,0.791510,0.777631,0.805838,0.791453,0.791773
6,cv std,0.018619,0.020088,0.017445,0.018240,0.018566


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_GaussianNB = GaussianNB()

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[449, 105],
        [101, 447]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8130671381950378


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8097826242446899 
Recall: 0.8156934380531311 
F1 Score: 0.8127272725105286


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8130813837051392


# Complement  Naive Bayes (ComplementNB)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_ComplementNB = ComplementNB()

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_ComplementNB, 'index', 5, False)

In [ ]:
val_acc

[0.8310679793357849,
 0.8132295608520508,
 0.7918288111686707,
 0.7879377603530884,
 0.7821011543273926]

In [ ]:
val_prec

[0.8167939186096191,
 0.8078431487083435,
 0.7865612506866455,
 0.7718631029129028,
 0.7713178396224976]

In [ ]:
val_rec

[0.8458498120307922,
 0.8142292499542236,
 0.7896825671195984,
 0.8055555820465088,
 0.7896825671195984]

In [ ]:
val_f1

[0.8310679793357849,
 0.8110235929489136,
 0.788118839263916,
 0.7883495092391968,
 0.7803921699523926]

In [ ]:
val_auroc

[0.8313218355178833,
 0.8132448196411133,
 0.79178786277771,
 0.7882739901542664,
 0.7822458744049072]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.8012330532073975
5-Fold CV Precision: 0.7908758521080017
5-Fold CV Recall: 0.8089999556541443
5-Fold CV F1 Score: 0.7997904181480407
5-Fold CV AUROC: 0.801374876499176


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.01824978437085815
5-Fold CV Precision: 0.018559372806591673
5-Fold CV Recall: 0.020704802066161418
5-Fold CV F1 Score: 0.018696963568070747
5-Fold CV AUROC: 0.018260222785614174


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-118-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-118-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.831068,0.816794,0.845850,0.831068,0.831322
1,2,0.813230,0.807843,0.814229,0.811024,0.813245
2,3,0.791829,0.786561,0.789683,0.788119,0.791788
3,4,0.787938,0.771863,0.805556,0.788350,0.788274
4,5,0.782101,0.771318,0.789683,0.780392,0.782246
5,cv mean,0.801233,0.790876,0.809000,0.799790,0.801375
6,cv std,0.018250,0.018559,0.020705,0.018697,0.018260


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
clf_ComplementNB = ComplementNB()

In [ ]:
trained_clf = clf_ComplementNB.fit(train_X, train_y)

In [ ]:
preds = trained_clf.predict(test_X)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[445, 109],
        [105, 443]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8058076500892639


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8025362491607666 
Recall: 0.8083941340446472 
F1 Score: 0.8054545521736145


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8058216571807861


# K-Nearest Neighbors (KNN)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_KNN = KNeighborsClassifier(n_neighbors=3)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_KNN, 'index', 5, True)

In [ ]:
val_acc

[0.7708737850189209,
 0.774319052696228,
 0.7782101035118103,
 0.7937743067741394,
 0.774319052696228]

In [ ]:
val_prec

[0.7922077775001526,
 0.7565543055534363,
 0.7827869057655334,
 0.8016529083251953,
 0.7809917330741882]

In [ ]:
val_rec

[0.7233201861381531,
 0.7984189987182617,
 0.7579365372657776,
 0.7698412537574768,
 0.75]

In [ ]:
val_f1

[0.7561983466148376,
 0.7769230604171753,
 0.7701612710952759,
 0.7854251265525818,
 0.7651821970939636]

In [ ]:
val_auroc

[0.7700570821762085,
 0.7746884226799011,
 0.7778232097625732,
 0.7933175563812256,
 0.7738549709320068]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7782992601394654
5-Fold CV Precision: 0.7828387260437012
5-Fold CV Recall: 0.7599033951759339
5-Fold CV F1 Score: 0.7707780003547668
5-Fold CV AUROC: 0.7779482483863831


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.008078450353846581
5-Fold CV Precision: 0.015073403126000702
5-Fold CV Recall: 0.02458653980928567
5-Fold CV F1 Score: 0.009965538031784174
5-Fold CV AUROC: 0.0080732044568377


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-140-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-140-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.770874,0.792208,0.723320,0.756198,0.770057
1,2,0.774319,0.756554,0.798419,0.776923,0.774688
2,3,0.778210,0.782787,0.757937,0.770161,0.777823
3,4,0.793774,0.801653,0.769841,0.785425,0.793318
4,5,0.774319,0.780992,0.750000,0.765182,0.773855
5,cv mean,0.778299,0.782839,0.759903,0.770778,0.777948
6,cv std,0.008078,0.015073,0.024587,0.009966,0.008073


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_KNN = KNeighborsClassifier(n_neighbors=3)

In [ ]:
trained_clf = clf_KNN.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[445, 109],
        [123, 425]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7894737124443054


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.795880138874054 
Recall: 0.775547444820404 
F1 Score: 0.7855822443962097


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7893983125686646


# Decision Tree (DT)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_DT = DecisionTreeClassifier(random_state=42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_DT, 'index', 5, True)

In [ ]:
val_acc

[0.7281553149223328,
 0.7334630489349365,
 0.7392995953559875,
 0.7470816969871521,
 0.7276264429092407]

In [ ]:
val_prec

[0.7306122183799744,
 0.7338709831237793,
 0.7218044996261597,
 0.7541666626930237,
 0.7314049601554871]

In [ ]:
val_rec

[0.7075098752975464,
 0.7193675637245178,
 0.761904776096344,
 0.7182539701461792,
 0.7023809552192688]

In [ ]:
val_f1

[0.718875527381897,
 0.7265468835830688,
 0.7413127422332764,
 0.7357723712921143,
 0.71659916639328]

In [ ]:
val_auroc

[0.727800726890564,
 0.7332470417022705,
 0.7397310137748718,
 0.74653160572052,
 0.7271446585655212]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7351252198219299
5-Fold CV Precision: 0.7343718647956848
5-Fold CV Recall: 0.7218834280967712
5-Fold CV F1 Score: 0.7278213381767273
5-Fold CV AUROC: 0.7348910093307495


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.007320611980635166
5-Fold CV Precision: 0.010704774989369252
5-Fold CV Recall: 0.021015024637946154
5-Fold CV F1 Score: 0.009516573079049503
5-Fold CV AUROC: 0.007374412683755974


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-164-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-164-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.728155,0.730612,0.707510,0.718876,0.727801
1,2,0.733463,0.733871,0.719368,0.726547,0.733247
2,3,0.739300,0.721804,0.761905,0.741313,0.739731
3,4,0.747082,0.754167,0.718254,0.735772,0.746532
4,5,0.727626,0.731405,0.702381,0.716599,0.727145
5,cv mean,0.735125,0.734372,0.721883,0.727821,0.734891
6,cv std,0.007321,0.010705,0.021015,0.009517,0.007374


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_DT = DecisionTreeClassifier(random_state=42)

In [ ]:
trained_clf = clf_DT.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[406, 148],
        [127, 421]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7504537105560303


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7398945689201355 
Recall: 0.7682482004165649 
F1 Score: 0.7538048624992371


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7505500912666321


# Random Forest (RF)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_RF, 'index', 5, True)

In [ ]:
val_acc

[0.7786407470703125,
 0.7587548494338989,
 0.7587548494338989,
 0.768482506275177,
 0.7607004046440125]

In [ ]:
val_prec

[0.7860082387924194,
 0.76113361120224,
 0.7622950673103333,
 0.7805907130241394,
 0.7632653117179871]

In [ ]:
val_rec

[0.7549406886100769,
 0.7430830001831055,
 0.738095223903656,
 0.7341269850730896,
 0.7420634627342224]

In [ ]:
val_f1

[0.7701612710952759,
 0.7519999742507935,
 0.75,
 0.7566462159156799,
 0.7525150775909424]

In [ ]:
val_auroc

[0.7782337665557861,
 0.7585146427154541,
 0.7583606243133545,
 0.7678269147872925,
 0.7603447437286377]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.76506667137146
5-Fold CV Precision: 0.7706585884094238
5-Fold CV Recall: 0.74246187210083
5-Fold CV F1 Score: 0.7562645077705383
5-Fold CV AUROC: 0.7646561384201049


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.0076767953814830744
5-Fold CV Precision: 0.010484207957217209
5-Fold CV Recall: 0.006997190147945432
5-Fold CV F1 Score: 0.00727673364129354
5-Fold CV AUROC: 0.007620307081813792


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-188-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-188-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.778641,0.786008,0.754941,0.770161,0.778234
1,2,0.758755,0.761134,0.743083,0.752000,0.758515
2,3,0.758755,0.762295,0.738095,0.750000,0.758361
3,4,0.768483,0.780591,0.734127,0.756646,0.767827
4,5,0.760700,0.763265,0.742063,0.752515,0.760345
5,cv mean,0.765067,0.770659,0.742462,0.756265,0.764656
6,cv std,0.007677,0.010484,0.006997,0.007277,0.007620


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
trained_clf = clf_RF.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[445, 109],
        [122, 426]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.790381133556366


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7962616682052612 
Recall: 0.7773722410202026 
F1 Score: 0.7867035865783691


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7903107404708862


# Support Vector Classification (SVC) -Linear Kernel-

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_SVC_L = SVC(kernel='linear', random_state=42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_SVC_L, 'index', 5, True)

In [ ]:
val_acc

[0.7825242877006531,
 0.7587548494338989,
 0.7626459002494812,
 0.7821011543273926,
 0.7470816969871521]

In [ ]:
val_prec

[0.8051947951316833,
 0.7433962225914001,
 0.7642276287078857,
 0.7991452813148499,
 0.7440000176429749]

In [ ]:
val_rec

[0.7351778745651245,
 0.7786561250686646,
 0.7460317611694336,
 0.7420634627342224,
 0.738095223903656]

In [ ]:
val_f1

[0.7685950398445129,
 0.760617733001709,
 0.7550200819969177,
 0.7695473432540894,
 0.7410358786582947]

In [ ]:
val_auroc

[0.7817110419273376,
 0.7590597867965698,
 0.7623288631439209,
 0.7813371419906616,
 0.7469102740287781]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7666215777397156
5-Fold CV Precision: 0.7711927890777588
5-Fold CV Recall: 0.7480048894882202
5-Fold CV F1 Score: 0.7589632153511048
5-Fold CV AUROC: 0.7662694215774536


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.013798636461649047
5-Fold CV Precision: 0.026450284019907105
5-Fold CV Recall: 0.01575706707276853
5-Fold CV F1 Score: 0.010435237771521957
5-Fold CV AUROC: 0.013474147585162953


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-212-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-212-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.782524,0.805195,0.735178,0.768595,0.781711
1,2,0.758755,0.743396,0.778656,0.760618,0.759060
2,3,0.762646,0.764228,0.746032,0.755020,0.762329
3,4,0.782101,0.799145,0.742063,0.769547,0.781337
4,5,0.747082,0.744000,0.738095,0.741036,0.746910
5,cv mean,0.766622,0.771193,0.748005,0.758963,0.766269
6,cv std,0.013799,0.026450,0.015757,0.010435,0.013474


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_SVC_L = SVC(kernel='linear', random_state=42)

In [ ]:
trained_clf = clf_SVC_L.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[440, 114],
        [109, 439]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7976406812667847


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7938517332077026 
Recall: 0.8010948896408081 
F1 Score: 0.7974568605422974


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7976593971252441


# Support Vector Classification (SVC) -Non-Linear Kernel-

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_SVC_RBF = SVC(kernel='rbf', random_state=42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_SVC_RBF, 'index', 5, True)

In [ ]:
val_acc

[0.784466028213501,
 0.7645914554595947,
 0.7762646079063416,
 0.7782101035118103,
 0.7548637986183167]

In [ ]:
val_prec

[0.8114035129547119,
 0.7538461685180664,
 0.7842323780059814,
 0.8026315569877625,
 0.7560975551605225]

In [ ]:
val_rec

[0.731225311756134,
 0.7747035622596741,
 0.75,
 0.726190447807312,
 0.738095223903656]

In [ ]:
val_f1

[0.7692307829856873,
 0.764132559299469,
 0.7667343020439148,
 0.762499988079071,
 0.7469879388809204]

In [ ]:
val_auroc

[0.7835515737533569,
 0.7647464275360107,
 0.7757633924484253,
 0.7772173285484314,
 0.7545438408851624]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7716791987419128
5-Fold CV Precision: 0.7816422343254089
5-Fold CV Recall: 0.7440429091453552
5-Fold CV F1 Score: 0.7619171142578125
5-Fold CV AUROC: 0.7711645126342773


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.010584663323949946
5-Fold CV Precision: 0.023486788302211542
5-Fold CV Recall: 0.017285477831057532
5-Fold CV F1 Score: 0.007807986481619653
5-Fold CV AUROC: 0.010282646594372951


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-236-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-236-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.784466,0.811404,0.731225,0.769231,0.783552
1,2,0.764591,0.753846,0.774704,0.764133,0.764746
2,3,0.776265,0.784232,0.750000,0.766734,0.775763
3,4,0.778210,0.802632,0.726190,0.762500,0.777217
4,5,0.754864,0.756098,0.738095,0.746988,0.754544
5,cv mean,0.771679,0.781642,0.744043,0.761917,0.771165
6,cv std,0.010585,0.023487,0.017285,0.007808,0.010283


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_SVC_RBF = SVC(kernel='rbf', random_state=42)

In [ ]:
trained_clf = clf_SVC_RBF.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[441, 113],
        [110, 438]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7976406812667847


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7949183583259583 
Recall: 0.7992700934410095 
F1 Score: 0.7970882654190063


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7976495027542114


# XGBoost

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_XGB = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_XGB, 'index', 5, True)

In [ ]:
val_acc

[0.7456310391426086,
 0.7509727478027344,
 0.7451362013816833,
 0.7587548494338989,
 0.7626459002494812]

In [ ]:
val_prec

[0.7606837749481201,
 0.747035562992096,
 0.7372549176216125,
 0.7689075469970703,
 0.7579365372657776]

In [ ]:
val_rec

[0.7035573124885559,
 0.747035562992096,
 0.7460317611694336,
 0.726190447807312,
 0.7579365372657776]

In [ ]:
val_f1

[0.7310061454772949,
 0.747035562992096,
 0.7416173815727234,
 0.7469387650489807,
 0.7579365372657776]

In [ ]:
val_auroc

[0.7449083924293518,
 0.7509124279022217,
 0.7451532483100891,
 0.7581334114074707,
 0.7625560164451599]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7526281476020813
5-Fold CV Precision: 0.7543636679649353
5-Fold CV Recall: 0.736150324344635
5-Fold CV F1 Score: 0.7449068784713745
5-Fold CV AUROC: 0.7523326992988586


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.007010292103744636
5-Fold CV Precision: 0.011051190211233164
5-Fold CV Recall: 0.01924720161514007
5-Fold CV F1 Score: 0.00874623389546048
5-Fold CV AUROC: 0.007026328974275421


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-260-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-260-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.745631,0.760684,0.703557,0.731006,0.744908
1,2,0.750973,0.747036,0.747036,0.747036,0.750912
2,3,0.745136,0.737255,0.746032,0.741617,0.745153
3,4,0.758755,0.768908,0.726190,0.746939,0.758133
4,5,0.762646,0.757937,0.757937,0.757937,0.762556
5,cv mean,0.752628,0.754364,0.736150,0.744907,0.752333
6,cv std,0.007010,0.011051,0.019247,0.008746,0.007026


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_XGB = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)

In [ ]:
trained_clf = clf_XGB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[439, 115],
        [114, 434]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7921960353851318


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.7905282378196716 
Recall: 0.7919707894325256 
F1 Score: 0.7912488579750061


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7921947836875916


# Multi-layer Perceptron (MLP)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_MLP = MLPClassifier(hidden_layer_sizes=(20, 20), activation='relu', alpha=1e-05, solver='adam', max_iter=1000, random_state=42)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_MLP, 'index', 5, True)

In [ ]:
val_acc

[0.753398060798645,
 0.7587548494338989,
 0.768482506275177,
 0.768482506275177,
 0.7373540997505188]

In [ ]:
val_prec

[0.762499988079071,
 0.7380073666572571,
 0.7628458738327026,
 0.7929515242576599,
 0.7368420958518982]

In [ ]:
val_rec

[0.7233201861381531,
 0.790513813495636,
 0.7658730149269104,
 0.7142857313156128,
 0.7222222089767456]

In [ ]:
val_f1

[0.7423934936523438,
 0.7633587718009949,
 0.7643564343452454,
 0.7515657544136047,
 0.7294589281082153]

In [ ]:
val_auroc

[0.7528814673423767,
 0.7592415809631348,
 0.7684326767921448,
 0.7674481868743896,
 0.737065315246582]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7572944045066834
5-Fold CV Precision: 0.7586293697357178
5-Fold CV Recall: 0.7432429909706115
5-Fold CV F1 Score: 0.7502266764640808
5-Fold CV AUROC: 0.7570138454437256


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.011534967618590524
5-Fold CV Precision: 0.020546158757512194
5-Fold CV Recall: 0.029745463389256203
5-Fold CV F1 Score: 0.013164750208094942
5-Fold CV AUROC: 0.011482581606926263


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-476-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-476-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.753398,0.762500,0.723320,0.742393,0.752881
1,2,0.758755,0.738007,0.790514,0.763359,0.759242
2,3,0.768483,0.762846,0.765873,0.764356,0.768433
3,4,0.768483,0.792952,0.714286,0.751566,0.767448
4,5,0.737354,0.736842,0.722222,0.729459,0.737065
5,cv mean,0.757294,0.758629,0.743243,0.750227,0.757014
6,cv std,0.011535,0.020546,0.029745,0.013165,0.011483


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
clf_MLP = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', alpha=1e-05, solver='adam', max_iter=1000, random_state=42)

In [ ]:
trained_clf = clf_MLP.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(test_y.to_numpy())

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[429, 125],
        [116, 432]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.7813067436218262


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1 = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1(preds, test_y).item())

Percision: 0.7755835056304932 
Recall: 0.7883211970329285 
F1 Score: 0.7819004654884338


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.7813447117805481
